# Proyecto de Modelación y Simulación

#### Se importan las librerías a utilizar.

In [13]:
import numpy as np
import random
import math

#### Se ingresa el sudoku que se desea resolver.

In [16]:
                   # 1  2  3  4  5  6  7  8  9
inicial = np.array([[1, 0, 3, 0, 6, 5, 0, 0, 0],   #1
                    [7, 0, 0, 0, 2, 0, 0, 0, 0],   #2
                    [5, 0, 0, 3, 0, 0, 0, 0, 0],   #3
                    [0, 0, 2, 6, 5, 0, 0, 3, 0],   #4
                    [0, 0, 1, 4, 3, 0, 6, 0, 0],   #5
                    [0, 0, 0, 0, 1, 7, 2, 0, 5],   #6
                    [0, 0, 0, 0, 0, 6, 0, 5, 0],   #7
                    [0, 0, 4, 0, 8, 0, 0, 6, 0],   #8
                    [0, 6, 0, 0, 4, 0, 0, 1, 0]])  #9

#### Se realiza una función que determina las posiciones (fila, columna) que son mutables, agrupando por subcuadro.

In [19]:
# Función que devuelve un arreglo de las posiciones mutables del sudoku.
def posiciones_mutables():
    mutables = [] # Lista donde se almacenan las posiciones mutables de cada subcuadrado.
    for fila in range(0, 9, 3): # Recorre las filas de todo el cuadro.
        for columna in range(0, 9, 3): # Recorre las columnas de todo el cuadro.
            posiciones = [] # Lista que almacena las posiciones mutables del subcuadrado analizado.
            for x in range(fila, fila + 3): # Recorre las filas del subcuadrado.
                for y in range(columna, columna + 3): # Recorre las columnas del subcuadrado.
                    if inicial[x, y] == 0: # Define si la celda se puede mutar.
                        posiciones.append((x, y))
            mutables.append(posiciones)
    return mutables # Lista con posiciones mutables.

mutables = posiciones_mutables() # Matriz con las posiciones mutables.

#### Se realiza una función que determina los números que faltan en un arreglo.

In [22]:
# Se establece un arreglo con los números que debe tener un arreglo del sudoku.
numeros = np.arange(1, 10)

# Función que devuelve una lista con los número que le faltan a un arreglo de entrada.
def numeros_faltantes(arreglo):
    return list(set(numeros) - set(arreglo)) # Se calcula la diferencia de conjuntos para determinar los faltantes.

# Se genera una lista con las listas de números faltantes en cada subcuadro.
faltantes = [numeros_faltantes(inicial[f:f+3, c:c+3].flatten()) for f in range(0, 9, 3) for c in range(0, 9, 3)]

### Funciones que generan la población inicial.

#### Se realiza una función que permite crear un individuo nuevo.

In [26]:
# Función que crea un nuevo individuo dejando invariantes las celdas iniciales.
def individuo_nuevo(ind):
    cont = 0 # Contador que permite recorrer las listas de faltantes de los sucuadros.
    for f in range(0, 9, 3): # Recorre las filas de todo el cuadro.
        for c in range(0, 9, 3): # Recorre las columnas de todo el cuadro.
            faltante = faltantes[cont][:] # Se define la lista de números del subcuadro.
            cont2 = 0 # Se define un contador que asegura que todos los fatantes serán usados.
            random.shuffle(faltante) # Se ordenan los faltantes de forma aleatoria.
            for x in range(f, f + 3): # Recorre las filas de los subcuadros.
                for y in range(c, c + 3): # Recorre las columnas de los subcuadros.
                    if inicial[x, y] == 0: # Condición de que falta un número.
                        ind[x, y] = faltante[cont2] # Se asigna un número faltante.
                        cont2 += 1
            cont += 1
    return ind.astype(int)

#### Se realiza una función que permite crear la población inicial.

In [29]:
# Función que develve una lista de tamaño n con nuevos individuos.
def poblacion_inicial(n):
    return [individuo_nuevo(inicial.copy()) for x in range(n)]

### Funciones Fitness.

#### Se realiza una función que determina la nota de un arreglo.

In [33]:
# Función que devuelve un número del 1 al 9 que indica la nota del arreglo.
def nota_arreglo(arreglo):
    return 9 - np.unique(arreglo).size # Es la cantidad de números repetidos en un arreglo.

#### Se realiza una función que determina la nota de un individuo.

In [36]:
# Función que determina la nota de un individuo, mediante las notas de sus columnas y fils.
def nota_individuo(ind):
    nota_neg = 0 # Almacena la nota que determina la cantidad de valores repetidos.
    nota_neg = sum(nota_arreglo(ind[:, j]) for j in range(9)) # La cantidad de valores repetidos por columna en todas las columnas.
    nota_neg += sum(nota_arreglo(ind[i, :]) for i in range(9)) # La cantidad de valores repetidos por fila en todas las filas.
    return 162 - nota_neg # Devuelve la nota del individuo.

#### Se realiza una función que determina las notas de la población.

In [39]:
# Función que devuelve una lista con las notas de cada individuo de la población.
def notas_individuos(poblacion):
    return [nota_individuo(x) for x in poblacion]

### Funciones de Selección.

#### Se realiza una función que determina los m individuos con notas más altas.

In [43]:
# Función que devuelve las posiciones correspondientes a las mejores notas.
def topm(notas, m):
    enumerada = list(enumerate(notas)) # lista de enumerados.
    enumerada.sort(key=lambda x: x[1], reverse=True) # Se ordena en función de los valores de las notas.
    return [idx for idx, valor in enumerada[:m]] # Se devuelve los m índices correspondientes a las notas más altas.

#### Se realiza una función que permite seleccionar individuos de la población.

In [46]:
# Función que selecciona individuos de la población.
def seleccion(poblacion, notas, m, torneo, elites, aleatorios):
    n = len(poblacion) # Número de individuos.
    mejores = int(m*elites) # Cantidad de mejores individuos a elegir.
    azar = int(m*aleatorios) # Cantidad de individuos al azar a elegir.
    torneos = m - mejores - azar # Cantidad de individuos elegidos por torneo.
    
    elegidos = [poblacion[i] for i in topm(notas, mejores)] # Elegidos por mejores notas.

    elegidos_azar = random.choices(poblacion, k=azar) # Elegidos al azar.
    for i in elegidos_azar:
        elegidos.append(i)

    for _ in range(torneos): # Elección por torneo.
        cand = random.sample(range(n), min(torneo, n)) # Se eligen candidatos al azar.
        ganador = max(cand, key=lambda i: notas[i]) # Elegimos al de mejor nota entre los candidatos.
        elegidos.append(poblacion[ganador])
        
    return elegidos # Listado de individuos elegidos.

### Funciones de cruce.

#### Se realiza una función que genera el cruce entre los individuos seleccionados.

In [50]:
# Función que permite cruzar los individuos seleccionados.
def cruzar(elegidos, n):
    m = len(elegidos) # Número de elegidos.
    hijos = [] # Lista donde se almacenará la nueva generación.
    
    for _ in range(n): # Ciclo que permite crear n individuos a partir del cruce.
        nuevo = np.zeros((9, 9), dtype = int) # 
        for f in range(0, 9, 3): # Recorre las filas de los subcuadros.
            for c in range(0, 9, 3): # Recorre las columnas de los subcuadros.
                nuevo[f:f+3, c:c+3] = random.choices(elegidos, k=1)[0][f:f+3, c:c+3] # Se asigna un subcuadro al hijo.
        hijos.append(nuevo)
    return hijos # Se devuelve los individuos de la nueva generación.

### Funciones de mutación.

#### Se realiza una función que permite mutar a un individuo.

In [54]:
# Función que permite mutar un individuo.
def mutar(individuo, d):
    for _ in range(d): # La cantidad de mutaciones requeridas.
        (f1, c1), (f2, c2) = random.sample(mutables[random.randint(0, 8)], 2) # Índices de las celdas a mutar.
        individuo[f1, c1], individuo[f2, c2] = individuo[f2, c2], individuo[f1, c1]  # Realiza el cambio de valores de celdas de un mismo subcuadro.
    return individuo # Devuelve al individuo mutado.

#### Se realiza una función que permite mutar a cada individuo de la población.

In [57]:
# Función que muta a todos los individuos de la población.
def mutar_poblacion(poblacion, d):
    for j in range(len(poblacion)): # Ciclo para recorrer a todos los individuos.
        poblacion[j] = mutar(poblacion[j], d) # Se muta al individuo.
    return poblacion # Se devuelve la población mutada.

### Algoritmo genético

In [60]:
def sudoku_genetico(n, m, d, max_gen = 5000, torneo = 3, elites = 0.6, aleatorios = 0.2):
    
    random.seed(10) # Semilla aleatoria.

    generacion, reinicio, maxima_nota, cambio_mutacion = 0, 0, 0, 1# Se definen contadores y variables almacenadoras de resultados.
    elit = int(m*elites) 
    reiniciar = int(max_gen*0.1)

    poblacion = poblacion_inicial(n) # Se realiza la primera iteración de la población.
    mutables = posiciones_mutables()
    notas = notas_individuos(poblacion)
    elegidos = seleccion(poblacion, notas, m, torneo, elites, aleatorios)

    mejor_nota = max(notas) # Se almacena las características de mejor individuo.
    mejor_ind = elegidos[0]


    while maxima_nota < 162 and generacion < max_gen: # Se recorren nuevas generaciones.

        mejores_ind =  elegidos[:elit] # Se crea la nueva generación.
        hijos = cruzar(elegidos, n - m)
        mutados = mutar_poblacion(hijos + elegidos[elit:], d)
        poblacion = mejores_ind + mutados
        notas = notas_individuos(poblacion)
        elegidos = seleccion(poblacion, notas, m, torneo, elites, aleatorios)
        
        if max(notas) > maxima_nota: # Se realizan cambios en caso de que se haya encontrado un nuevo mejor individuo.
            maxima_nota = max(notas)
            mejor_ind = elegidos[0].copy()
            cambio_mutacion, reinicio = 1, 0
            print(generacion, maxima_nota)
            
        if reinicio == reiniciar: # En caso de que no se obtengan cambios se realiza una mutación completa a todos los individuos.
            poblacion = mutar_poblacion(poblacion, cambio_mutacion * d)
            cambio_mutacion += 1
            reinicio = 0
        
        generacion += 1 # Iteradores.
        reinicio +=1 
        
    return generacion, mejor_ind, maxima_nota # Se devuelve la generación, el mejor individuo y su nota.

### Generación de la solución

#gen, sol_mio, nota = sudoku_genetico(n = 100, m = 15, d = 2, max_gen = 5000, torneo = 3, elites = 0.3, aleatorios = 0.1)
gen, sol_mio, nota = sudoku_genetico(n = 150, m = 40, d = 2, max_gen = 5000, torneo = 4, elites = 0.3, aleatorios = 0.4)
gen, sol_mio, nota

### Corroboración

In [67]:
               # 1  2  3  4  5  6  7  8  9
sol = np.array([[1, 2, 3, 8, 6, 5, 4, 9, 7], # 1
                [7, 4, 6, 1, 2, 9, 5, 8, 3], # 2
                [5, 8, 9, 3, 7, 4, 1, 2, 6], # 3
                [4, 7, 2, 6, 5, 8, 9, 3, 1], # 4
                [9, 5, 1, 4, 3, 2, 6, 7, 8], # 5
                [6, 3, 8, 9, 1, 7, 2, 4, 5], # 6
                [8, 1, 7, 2, 9, 6, 3, 5, 4], # 7
                [3, 9, 4, 5, 8, 1, 7, 6, 2], # 8
                [2, 6, 5, 7, 4, 3, 8, 1, 9]]) # 9

In [69]:
sol_mio - sol

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

CasoI:reducción de la población (n) con el número de elegidos m=15

n=100

In [90]:
gen, sol_mio, nota = sudoku_genetico(n = 100, m = 15, d = 2, max_gen = 5000, torneo = 3, elites = 0.3, aleatorios = 0.1)

0 129
1 136
3 138
4 139
5 140
6 141
7 142
12 143
14 144
15 146
21 147
32 148
33 149
39 151
45 152
48 153
54 154
63 156
124 157
130 158
202 159
352 160
901 162


n=50

In [73]:
gen, sol_mio, nota = sudoku_genetico(n = 50, m = 15, d = 2, max_gen = 5000, torneo = 3, elites = 0.3, aleatorios = 0.1)

0 132
1 136
3 138
6 139
7 140
8 141
9 143
11 144
13 145
19 147
27 150
43 151
73 152
81 153
159 154
166 155
281 156
1358 158
3352 159
3923 160


n=30

In [93]:
gen, sol_mio, nota = sudoku_genetico(n = 30, m = 15, d = 2, max_gen = 5000, torneo = 3, elites = 0.3, aleatorios = 0.1)

0 131
1 132
2 137
3 138
9 140
11 142
13 143
25 144
28 146
44 149
235 150
240 151
667 152
711 153
3447 154


n=20

In [96]:
gen, sol_mio, nota = sudoku_genetico(n = 20, m = 15, d = 2, max_gen = 5000, torneo = 3, elites = 0.3, aleatorios = 0.1)

0 126
1 129
3 130
4 131
5 133
8 134
12 135
13 137
22 138
39 139
43 141
74 142
101 144
1651 145
4604 146


n=10

In [99]:
gen, sol_mio, nota = sudoku_genetico(n = 10, m = 15, d = 2, max_gen = 5000, torneo = 3, elites = 0.3, aleatorios = 0.1)

0 126
5 127
44 128
63 129
109 130
800 134
2817 137
